## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(15)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Таблица содержит 12 столбцов с информацией по каждому заемщику.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#print(data.isnull().sum())
#print(data[data['total_income'].isnull()].head())
#print(data.isnull().sum())
#print(data[data['total_income'].isnull()].head())
data['days_employed'] = data['days_employed'].apply(abs)
days_employed_mean = data['days_employed'].mean()
data['days_employed'] = data['days_employed'].fillna(value=days_employed_mean )

data['total_income'] = data['total_income'].apply(abs)
total_income_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(value=total_income_median )


inc_type = data['income_type'].unique()
for types in inc_type:
    data.loc[data['income_type'] == types,'total_income']=data.loc[data['income_type'] == types,'total_income'].fillna(data[data['income_type'] == types]['total_income'].median())

#print(data['children'].unique())
#print(data['dob_years'].unique())
data['children'] = data['children'].apply(abs)
#print(data['dob_years'].isnull().count())
dob_years_mean = data['dob_years'].mean()
data['dob_years'] = data['dob_years'].replace('0', dob_years_mean)
data.head(15)






,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

1)Были обнаружены пропущенные значения в столбцах: days_employed, total_income 
2)Пропуски в этих столбцах могут быть связаны с тем, что в данный момент человек является безработным на сегодняшний день
3)Отсутствие оффициальной занятости не обязательно означает, что заемщик неплатежеспособен, поэтому я заменил пропуски средними значениями по столбцу

### Замена типа данных

In [4]:
#data.info()
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод 

Замена типа данных в соответствующих столбцах облегчит их анализ для последующих целей

### Обработка дубликатов

In [5]:
#data.info()
#data.duplicated().sum()
#data['education'].unique()
data['education'] = data['education'].str.lower()
data.head(20)
data = data.drop_duplicates()
data.duplicated().sum()

0

### Вывод

Сначала подсчитана сумма дубликатов по всей базе данных, затем проведен поиск уникальных значений для конкретного столбца. Столбец с информацией по образованию заемщика приведен к унифицированному виду. Дубликаты в этом случае могли появиться из-за разных методов ввода: с клавиатуры компьютера и мобильного телефона, нампример,при включенном Caps Lock, также на некоторых смартфонах по умолчанию установлена печать с заглавной буквы.

In [6]:
data.duplicated().sum()

0

### Лемматизация

In [7]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize)
all_lemmas_list = data['purpose_lemma'].values
flat_lemmas_list = []
for sublist in all_lemmas_list:
  for item in sublist:
      flat_lemmas_list.append(item)
#print(flat_lemmas_list)

def lemma_purpose(row):
    lemma_purposes = m.lemmatize(row['purpose'])
    if 'недвижимость' in lemma_purposes:
        return 'недвижимость'
    if 'свадьба' in lemma_purposes:
        return 'свадьба'
    if 'автомобиль' in lemma_purposes:
        return 'автомобиль'
    if 'образование' in lemma_purposes:
        return 'образование'
    if 'операция' in lemma_purposes:
        return 'операция'
    return 'что_то_другое'
 
data['purpose_classified'] = data.apply(lemma_purpose, axis=1)
print(data['purpose_classified'].unique())

['что_то_другое' 'автомобиль' 'образование' 'свадьба' 'операция'
 'недвижимость']


### Вывод

Для процедуры лемматизации данных в первую очередь был проведен анализ значений, встречающихся в столбце целей получения займа. Следующим шагом я разбил полученные значения на несколько категорий для их единообразия. Это позволит мне быстрее и проще анализировать заемщиков по причине займа.

### Категоризация данных

In [8]:
print(data['total_income'].describe())
def income_group(income):
        """
         
        """

        if  income <= 1.076230e+05:
                return 'низкий доход'
        if 1.076230e+05 <= income < 1.560435e+05:
                return 'умеренный доход'
        if 1.560435e+05 <=income < 1.958132e+05:
                return 'средний доход'
        return 'высокий доход'

data['income_gradation'] = data['total_income'].apply(income_group)
print(data['income_gradation'].value_counts())

count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64
умеренный доход    7466
высокий доход      5364
низкий доход       5364
средний доход      3260
Name: income_gradation, dtype: int64


### Вывод

Для этого набора данных был выделен словарь сгруппированный по доходам заемщиков.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
data_pivot_children = data.pivot_table( columns = 'children',
values = 'debt', aggfunc = 'count')
data_pivot_children_sum = data.pivot_table(index='children',aggfunc='sum')['debt']
print(data_pivot_children)
print(data_pivot_children_sum/data_pivot_children*100)
# здесь мне показалось, что нет необходимости находить долю от всех заемщиков с детьми/без, я прав?

children     0     1     2    3   4   5   20
debt      14091  4855  2052  330  41   9  76
children        0         1         2         3         4    5          20
debt      7.543822  9.165808  9.454191  8.181818  9.756098  0.0  10.526316


In [10]:
data.pivot_table(index='children',aggfunc='sum')['debt']

children
0     1063
1      445
2      194
3       27
4        4
5        0
20       8
Name: debt, dtype: int64

### Вывод

Исходя из сводных данных можно сделать вывод, что заемщики без детей более ответственно относятся к своевременным выплатам займов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [49]:
data_pivot_marriage = data.pivot_table( columns = 'family_status',
values = 'debt', aggfunc = 'sum')
print(data_pivot_marriage)

data_family = [274/(data[data['family_status']== 'Не женат / не замужем']['family_status']).count()*100 ,
               85/(data[data['family_status']== 'в разводе']['family_status']).count()*100 ,
               63/(data[data['family_status']== 'вдовец / вдова']['family_status']).count()*100 ,
               388/(data[data['family_status']== 'гражданский брак']['family_status']).count()*100,
               931/(data[data['family_status']== 'женат / замужем']['family_status']).count()*100]
data_columns = ['Не женат / не замужем',  'в разводе',  'вдовец / вдова',  'гражданский брак',  'женат / замужем'  ]
data_check = pd.DataFrame([data_family], columns = data_columns)
data_check


family_status  Не женат / не замужем  в разводе  вдовец / вдова  \
debt                             274         85              63   

family_status  гражданский брак  женат / замужем  
debt                        388              931  


,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
0,9.75089,7.112971,6.569343,9.347145,7.545182


### Вывод

Исходя из сводных данных можно сделать вывод, что овдовевшие и разведенный заемщики более ответственно относятся к своевременным выплатам займов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [45]:
data_pivot_income = data.pivot_table( columns = 'income_gradation',
values = 'debt', aggfunc = 'sum')
print(data_pivot_income)
data_income = [383/(data[data['income_gradation']== 'высокий доход']['income_gradation']).count()*100 ,
               427/(data[data['income_gradation']== 'низкий доход']['income_gradation']).count()*100 ,
               293/(data[data['income_gradation']== 'средний доход']['income_gradation']).count()*100 ,
               638/(data[data['income_gradation']== 'умеренный доход']['income_gradation']).count()*100]
data_columns = ['высокий доход' ,'низкий доход', 'средний доход', 'умеренный доход' ]
data_check = pd.DataFrame([data_income], columns = data_columns)
data_check




income_gradation  высокий доход  низкий доход  средний доход  умеренный доход
debt                        383           427            293              638


,высокий доход,низкий доход,средний доход,умеренный доход
0,7.140194,7.960477,8.98773,8.545406


### Вывод

Исходя из сводных данных можно сделать вывод по отношению несвоевременных выплат займа к общему количеству заемщиков в каждой группе.Результат выборки показывает нам, что заемщики с высоким и низким доходом более ответственно относятся
в выплате займа в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
data_pivot_income = data.pivot_table( columns = 'purpose_classified',
values = 'debt', aggfunc = 'sum')
print(data_pivot_income)

data_purpose = [403/(data[data['purpose_classified']== 'автомобиль']['purpose_classified']).count()*100 ,
                474/(data[data['purpose_classified']== 'недвижимость']['purpose_classified']).count()*100 ,
                370/(data[data['purpose_classified']== 'образование']['purpose_classified']).count()*100 ,
                48/(data[data['purpose_classified']== 'операция']['purpose_classified']).count()*100,
                186/(data[data['purpose_classified']== 'свадьба']['purpose_classified']).count()*100,
                260/(data[data['purpose_classified']== 'что_то_другое']['purpose_classified']).count()*100]
data_columns = ['автомобиль' , 'недвижимость' , 'образование'  ,'операция' , 'свадьба', 'что_то_другое']
data_check = pd.DataFrame([data_purpose], columns = data_columns)
data_check


purpose_classified  автомобиль  недвижимость  образование  операция  свадьба  \
debt                       403           474          370        48      186   

purpose_classified  что_то_другое  
debt                          260  


,автомобиль,недвижимость,образование,операция,свадьба,что_то_другое
0,9.359034,7.463392,9.220035,7.361963,8.003442,6.827731


### Вывод

Исходя из сводных данных можно сделать вывод, что заемщики с целями кредита на недвижимость, операции и другие потребности, чаще не имеют задолженности по выплатам .

### Шаг 4. Общий вывод

Получив данные от заказчика я провел предобработку данных и ответил на вопросы по информации о заемщиках.

Были рассмотренны такие свойства заемщиков как причина займа, количество детей, доход, семейное положение и их влияние на успешную выплату задолженностей.


Краткие выводы по полученным данным:

1) Заемщики без детей более ответственно относятся к своевременным выплатам займов.

2) Овдовевшие, разведенный и женатые/замужние заемщики более ответственно относятся к своевременным выплатам займов.

3) Заемщики с высоким и низким доходом более склонны к  своевременной выплате займа.

4) Заемщики , чей целью получения кредита являются недвижимость, операции и другие потребности, чаще не имеют задолженности по выплатам.